## TFM-17HB103

Total stream flow model that uses two stress models: one for recharge and one for groundwater head.

Joaquim Altimiras Granel

Import libraries

In [ ]:
# Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pastas as ps
import os

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score

Load all input data

In [ ]:
# LOAD FLOW DATA (OSERIES)

# Load flow data
path_baseflow = input('Enter path for stream level:')
baseflow = pd.read_excel(path_baseflow, usecols=[0, 1], index_col=0, parse_dates=True)

# Trim the series at start and end
# Remove initial rows with missing values
baseflow = baseflow.dropna(subset=[baseflow.columns[0]]).copy()

# Remove initial rows with missing values
while baseflow.iloc[0].isna().any():
    baseflow = baseflow.iloc[1:]

# Remove trailing rows with missing values
while baseflow.iloc[-1].isna().any():
    baseflow = baseflow.iloc[:-1]

# Display the new start and end date of the series after trimming
new_start_date = baseflow.index.min()
new_end_date = baseflow.index.max()
print(f"Start date of the series after trimming: {new_start_date}")
print(f"End date of the series after trimming: {new_end_date}")

# Check for time gaps
# Ensure data frame is complete for each day in the range
complete_index = pd.date_range(start=new_start_date, end=new_end_date, freq='D')
baseflow = baseflow.reindex(complete_index)

# Check for time gaps now represented as NaNs due to reindexing
missing_dates_count = baseflow.isna().sum().sum()  # Count all NaNs which represent missing dates
print(f"Number of missing dates (time gaps) added: {missing_dates_count}")

# Check for missing values and interpolate
# Check for missing values within the new timeframe and interpolate
missing_values_count_before = baseflow.isna().sum().sum()  # Total NaN count before interpolation
baseflow.interpolate(method='linear', inplace=True)
missing_values_count_after = baseflow.isna().sum().sum()  # Total NaN count after interpolation

# Calculate and display how many values have been interpolated
values_interpolated = missing_values_count_before - missing_values_count_after
print(f"Number of values interpolated: {values_interpolated}")


# LOAD GROUNDWATER LEVEL DATA

# Load groundwater level data
path_gw = input('Enter path for gw_level:')
gw_level = pd.read_excel(path_gw, usecols=[0, 1], index_col=0, parse_dates=True)


# Trim the series at start and end
# Remove initial rows with missing values
gw_level = gw_level.dropna(subset=[gw_level.columns[0]]).copy()

# Remove initial rows with missing values
while gw_level.iloc[0].isna().any():
    gw_level = gw_level.iloc[1:]

# Remove trailing rows with missing values
while gw_level.iloc[-1].isna().any():
    gw_level = gw_level.iloc[:-1]

# Display the new start and end date of the series after trimming
new_start_date = gw_level.index.min()
new_end_date = gw_level.index.max()
print(f"Start date of the series after trimming: {new_start_date}")
print(f"End date of the series after trimming: {new_end_date}")

# Check for time gaps
# Ensure data frame is complete for each day in the range
complete_index = pd.date_range(start=new_start_date, end=new_end_date, freq='D')
gw_level = gw_level.reindex(complete_index)

# Check for time gaps now represented as NaNs due to reindexing
missing_dates_count = gw_level.isna().sum().sum()  # Count all NaNs which represent missing dates
print(f"Number of missing dates (time gaps) added: {missing_dates_count}")

# Check for missing values and interpolate
# Check for missing values within the new timeframe and interpolate
missing_values_count_before = gw_level.isna().sum().sum()  # Total NaN count before interpolation
gw_level.interpolate(method='linear', inplace=True)
missing_values_count_after = gw_level.isna().sum().sum()  # Total NaN count after interpolation

# Calculate and display how many values have been interpolated
values_interpolated = missing_values_count_before - missing_values_count_after
print(f"Number of values interpolated: {values_interpolated}")


# LOAD PRECIPITATION DATA

# Precipitation (Read it as m/day)
file_path=input("Enter file path for precipitation data: ") 
prec = pd.read_excel(file_path, parse_dates=True, index_col=0, names=['prec']).squeeze()
prec = prec.to_frame()

# Trim the series at start and end
# Remove initial rows with missing values
prec = prec.dropna(subset=[prec.columns[0]]).copy()

# Remove initial rows with missing values
while prec.iloc[0].isna().any():
    prec = prec.iloc[1:]

# Remove trailing rows with missing values
while prec.iloc[-1].isna().any():
    prec = prec.iloc[:-1]

# Display the new start and end date of the series after trimming
new_start_date = prec.index.min()
new_end_date = prec.index.max()
print(f"Start date of the series after trimming: {new_start_date}")
print(f"End date of the series after trimming: {new_end_date}")

# Check for time gaps
# Ensure data frame is complete for each day in the range
complete_index = pd.date_range(start=new_start_date, end=new_end_date, freq='D')
prec = prec.reindex(complete_index)

# Check for time gaps now represented as NaNs due to reindexing
missing_dates_count = prec.isna().sum().sum()  # Count all NaNs which represent missing dates
print(f"Number of missing dates (time gaps) added: {missing_dates_count}")

# Check for missing values and interpolate
# Check for missing values within the new timeframe and interpolate
missing_values_count_before = prec.isna().sum().sum()  # Total NaN count before interpolation
prec.interpolate(method='linear', inplace=True)
missing_values_count_after = prec.isna().sum().sum()  # Total NaN count after interpolation

# Calculate and display how many values have been interpolated
values_interpolated = missing_values_count_before - missing_values_count_after
print(f"Number of values interpolated: {values_interpolated}")


# LOAD TEMPERATURE DATA

# Enter path
file_path = input("Enter file path for temperature data: ")

# Read full file
temp = pd.read_excel(file_path, index_col=0, parse_dates=True)
temp = temp.squeeze()


# LOAD PET-DATA

file_path=input("Enter the path to the evaporation data: ")

# Read the whole file
evap = pd.read_excel(file_path, parse_dates=True, index_col=0)

Display all loaded input time series

In [ ]:
# Display start dates

baseflow_start_date = baseflow.index.min()
print(f"Start date of baseflow after trimming: {baseflow_start_date}")

gw_level_start_date = gw_level.index.min()
print(f"Start date of gw_level after trimming: {gw_level_start_date}")

prec_start_date = prec.index.min()
print(f"Start date of prec after trimming: {prec_start_date}")

#evap_selected_start_date = evap_selected.index.min()
#print(f"Start date of evap_selected after trimming: {evap_selected_start_date}")

temp_start_date =temp.index.min()
print(f"Start date of temp (no trimming/interpolation): {temp_start_date}")


# Plot all input data

# Plot for 'baseflow' DataFrame
plt.figure(figsize=(10, 2))  # Set the figure size for better visibility
plt.plot(baseflow.index, baseflow.iloc[:, 0], label='Baseflow Data', color='black')  # Plot the first column
plt.title('Baseflow Time Series')  # Title of the plot
plt.xlabel('Date')  # Label for the x-axis
plt.ylabel('Baseflow')  # Label for the y-axis
plt.grid(True)  # Show grid
plt.show()  # Display the plot

# Plot for 'gw_level' DataFrame
plt.figure(figsize=(10, 2))  # Set the figure size for better visibility
plt.plot(gw_level.index, gw_level.iloc[:, 0], label='Groundwater Level Data', color='black')  # Plot the first column
plt.title('Groundwater Level Time Series')  # Title of the plot
plt.xlabel('Date')  # Label for the x-axis
plt.ylabel('Groundwater Level')  # Label for the y-axis
plt.grid(True)  # Show grid
plt.show()  # Display the plot

# Plot for 'prec' DataFrame (Precipitation Data)
plt.figure(figsize=(10, 2))  # Set the figure size for better visibility
plt.plot(prec.index, prec.iloc[:, 0], label='Precipitation Data', color='black')  # Plot the first column
plt.title('Precipitation Time Series')  # Title of the plot
plt.xlabel('Date')  # Label for the x-axis
plt.ylabel('Precipitation (mm)')  # Label for the y-axis
plt.grid(True)  # Show grid
plt.show()  # Display the plot

# Plot for 'evap_selected' DataFrame (Evaporation Data)
#plt.figure(figsize=(10, 2))  # Set the figure size for better visibility
#plt.plot(evap_selected.index, evap_selected.iloc[:, 0], label='Evaporation Data', color='black')  # Plot the first column
#plt.title(f'Evaporation Time Series - {evap_choice}')  # Title of the plot
#plt.xlabel('Date')  # Label for the x-axis
#plt.ylabel('Evaporation (mm)')  # Label for the y-axis
#plt.grid(True)  # Show grid
#plt.show()  # Display the plot

# Plot for 'temp' DataFrame (Temperature Data)
plt.figure(figsize=(10, 2))  # Set the figure size for better visibility
plt.plot(temp, label='Temperature Data', color='black')  # Plot the first column
plt.title('Temperature Time Series')  # Title of the plot
plt.xlabel('Date')  # Label for the x-axis
plt.ylabel('Temperature (Celsius)')  # Label for the y-axis
plt.grid(True)  # Show grid
plt.show()  # Display the plot

Extra plot, not necessary

In [ ]:
# Plotting flow/level and gw_level

# Specify the start and end dates for the plot
start_date = '2022-12-01'
end_date = '2023-12-01'

# Filter data based on the start and end date
baseflow_filtered = baseflow[start_date:end_date]
gw_level_filtered = gw_level[start_date:end_date]

# Create a plot
fig, ax1 = plt.subplots(figsize=(10, 5))

# Plotting baseflow on the primary y-axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Baseflow', color=color)
ax1.plot(baseflow_filtered.index, baseflow_filtered, color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for the groundwater level
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Groundwater Level', color=color)  
ax2.plot(gw_level_filtered.index, gw_level_filtered, color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Title and grid
plt.title('Baseflow and Groundwater Level Over Time')
plt.grid(True)

# Show the plot
plt.show()

## Model creation

In [ ]:
# Time range for calibration

tmin = '2022-12-01'
tmax = '2023-12-01'

In [ ]:
# Create all PET-models

# Time period definitions
tmin_cal = '2022-12-01'
tmax_cal = '2023-12-01'
tmin_val = pd.Timestamp("2023-12-01")
tmax_val = pd.Timestamp("2024-04-01")

# Extended period for the entire R² calculation
tmin_ext = tmin_cal
tmax_ext = tmax_val

# Create models
models = []  # Store models for later use or analysis
pet_method_names = evap.columns.tolist()  # Extract PET method names

for i, method_name in enumerate(pet_method_names):  # Iterate over each evapotranspiration series
    evap_selected = evap.iloc[:, i].squeeze()

    # Naming the model after the PET method
    model_name = f"model_{method_name}"
    ml = ps.Model(baseflow, name=model_name)
    
    # Add stress model 1 for recharge
    sm_rch = ps.RechargeModel(
        prec=prec,
        temp=temp,
        evap=evap_selected,
        rfunc=ps.Gamma(),
        name=f"rch_{method_name}",
        recharge=ps.rch.FlexModel(gw_uptake=True, snow=True)
    )
    ml.add_stressmodel(sm_rch)

    # Add stress model 2 for groundwater levels
    sm_gw = ps.StressModel(
        gw_level,
        rfunc=ps.Gamma(),
        name=f"sm_gw_{method_name}"
    )
    ml.add_stressmodel(sm_gw)
    
    # Adjust parameters
    ml.set_parameter(f"rch_{method_name}_kv", vary=True)
    
    # Solve the model in two steps
    ml.solve(tmin=tmin_cal, tmax=tmax_cal, noise=False, fit_constant=False, report=False, solver=ps.LeastSquares())
    ml.set_parameter(f"rch_{method_name}_srmax", vary=False)
    ml.solve(tmin=tmin_cal, tmax=tmax_cal, noise=True, fit_constant=False, initial=False, report=False, solver=ps.LeastSquares())
    
    models.append(ml)

In [ ]:
# Plot all PET-model results

# Prepare DataFrame for R² values
r2_results = pd.DataFrame(columns=["Model", "R2_Calibration", "R2_Validation", "R2_Extended"])

# Setup figure
fig, ax = plt.subplots(figsize=(12, 7))

# Color palette to maintain consistency
colors = plt.cm.viridis(np.linspace(0, 1, len(models)))

# Loop through models for simulation and plotting
for idx, model in enumerate(models):
    # Calibration period
    sim_cal = model.simulate(tmin=tmin_cal, tmax=tmax_cal)
    obs_cal = baseflow[tmin_cal:tmax_cal]
    r2_cal = r2_score(obs_cal, sim_cal)

    # Validation period
    sim_val = model.simulate(tmin=tmin_val, tmax=tmax_val)
    obs_val = baseflow[tmin_val:tmax_val]
    r2_val = r2_score(obs_val, sim_val)

    # Extended period
    sim_ext = model.simulate(tmin=tmin_ext, tmax=tmax_ext)
    obs_ext = baseflow[tmin_ext:tmax_ext]
    r2_ext = r2_score(obs_ext, sim_ext)

    # Store R² results using pd.concat()
    r2_row = pd.DataFrame([{
        "Model": model.name, 
        "R2_Calibration": r2_cal, 
        "R2_Validation": r2_val,
        "R2_Extended": r2_ext
    }])
    r2_results = pd.concat([r2_results, r2_row], ignore_index=True)
    
    # Plotting results
    ax.plot(sim_cal.index, sim_cal, label=f"{model.name}", color=colors[idx])
    ax.plot(sim_val.index, sim_val, linestyle='--', color=colors[idx])  # Same color for consistency

# Plot observed data
ax.plot(obs_cal.index, obs_cal, color='black', linewidth=3, label="Observed - Calibration")
ax.plot(obs_val.index, obs_val, color='black', linewidth=3, label="Observed - Validation")

# Add a vertical line to mark the transition from calibration to validation
ax.axvline(x=pd.Timestamp("2023-12-01"), color='red', linestyle=':', linewidth=2, label='Start of Validation')

# Formatting plot
ax.set_title('Model Simulations across Calibration and Validation Periods')
ax.set_xlabel('Date')
ax.set_ylabel('Flow')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
ax.grid(True)

plt.tight_layout()  # Adjust layout
plt.show()

# Print the table of R2 results with formatted decimal places
print("R² values for each model across Calibration, Validation, and Extended periods:")
pd.options.display.float_format = '{:,.3f}'.format  # Formatting float to three decimals
print(r2_results)

## Choosing specific model

In [ ]:
# Plot specific model

# Function to plot the results for a specified model, including observations
def plot_model_results(models, method_name, baseflow):
    # Find the model with the specified method name
    model = next((m for m in models if method_name in m.name), None)
    if model is None:
        print("Model not found.")
        return

    # Define time periods for calibration and validation
    tmin_cal = '2022-12-01'
    tmax_cal = '2023-12-01'
    tmin_val = '2023-12-01'
    tmax_val = '2024-04-01'
    
    # Simulate for calibration and validation periods
    sim_cal = model.simulate(tmin=tmin_cal, tmax=tmax_cal)
    sim_val = model.simulate(tmin=tmin_val, tmax=tmax_val)
    
    # Extract the observed data for calibration and validation periods
    obs_cal = baseflow[tmin_cal:tmax_cal]
    obs_val = baseflow[tmin_val:tmax_val]

    # Plot the results
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(obs_cal.index, obs_cal, label='Observed - Calibration', color='black')
    ax.plot(obs_val.index, obs_val, label='Observed - Validation', color='black')
    ax.plot(sim_cal.index, sim_cal, label='Simulated - Calibration', color='blue')
    ax.plot(sim_val.index, sim_val, label='Simulated - Validation', color='green', linestyle='--')

    # Add a vertical line to mark the transition from calibration to validation
    ax.axvline(x=pd.Timestamp("2023-12-01"), color='red', linestyle=':', linewidth=2)

    # Formatting plot
    ax.set_title(f'Results for Model: {model.name}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Stream Flow')
    ax.legend()
    ax.grid(True)
    plt.show()

# Example usage
method_name = input("Enter the model method name to plot: ")
plot_model_results(models, method_name, baseflow)

## Show calibration results

In [ ]:
# Display method names

for i in range(1, 19):
    print(f"Method {i}: {evap.columns[i-1]}")

Choose PET method

In [ ]:
# Choice of PET method to use for displaying calibration results

evap_choice = input("Type the name of the method that should be used: ")

# Access the actual Series based on user input
if evap_choice in evap:
    evap_selected = evap[evap_choice]
else:
    print(f"Method {evap_choice} not found. Please check your input.")
    # You might want to handle this situation more gracefully, e.g., asking for input again or exiting the script

In [ ]:
# Flow Model

# Time frame calibration

tmin = '2022-12-01'
tmax = '2023-12-01'

model = ps.Model(baseflow, name = "Model")

# Add stress model 1
sm_rch = ps.RechargeModel(
    prec=prec,
    temp=temp,
    evap=evap_selected,
    rfunc=ps.Gamma(),
    name="sm_rch",
    recharge=ps.rch.FlexModel(gw_uptake=True, snow=True)
)
model.add_stressmodel(sm_rch)

# Add stress model 2
sm_gw = ps.StressModel(
    gw_level,
    rfunc=ps.Gamma(),
    name="sm_gw"
)
model.add_stressmodel(sm_gw)

# SOLVER (TWO-STEP)

# As the evaporation used is a very rough estimation, vary k_v
model.set_parameter("sm_rch_kv", vary=True)

# Step 1
model.solve(
    tmin=tmin,
    tmax=tmax,
    noise=False,
    fit_constant=False,
    report=False,
    solver=ps.LeastSquares()
)

model.set_parameter("sm_rch_srmax", vary=False)

# Step 2
model.solve(
    tmin=tmin,
    tmax=tmax,
    noise=True,
    fit_constant=False,
    initial=False,
    report=False,
    solver=ps.LeastSquares()
)

model.plots.results(figsize=(12, 10))